# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 66 new papers today
          22 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/21 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2211.04472


extracting tarball to tmp_2211.04472...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.04474


/tmp/ipykernel_2293/4030337529.py:34: LatexWarning: 2211.04474 did not run properly
not a gzip file
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.04937


extracting tarball to tmp_2211.04937...

 done.
Retrieving document from  https://arxiv.org/e-print/2211.05022


extracting tarball to tmp_2211.05022...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.05022-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.05022) | **PRODIGE -- Envelope to Disk with NOEMA II. Small-scale temperature  structure and a streamer feeding the SVS13A protobinary using CH3CN and DCN**  |
|| T.-H. Hsieh, et al. -- incl., <mark>Th. Henning</mark>, <mark>G. V. Smirnov-Pinchukov</mark>, <mark>D. Semenov</mark>, <mark>B. Zhao</mark> |
|*Appeared on*| *2022-11-10*|
|*Comments*| *20 pages, 19 figures, accepted to A&A*|
|**Abstract**| Aims. We present high sensitivity and high-spectral resolution NOEMA observations of the Class 0/I binary system SVS13A, composed of the low-mass protostars VLA4A and VLA4B with a separation of ~90 au. VLA4A is undergoing an accretion burst that enriches the chemistry of the surrounding gas. This gives us an excellent opportunity to probe the chemical and physical conditions as well as the accretion process. Methods. We observe the (12K-11K) lines of CH3CN and CH313CN, the DCN (3-2) line, and the C18O (2-1) line toward SVS13A using NOEMA. Results. We find complex line profiles at disk scales which cannot be explained by a single component or pure Keplerian motion. By adopting two velocity components to model the complex line profiles, we find that the temperatures and densities are significantly different between these two components. This suggests that the physical conditions of the emitting gas traced via CH3CN can change dramatically within the circumbinary disk. In addition, combining our observations of DCN (3-2) with previous ALMA high-angular-resolution observations, we find that the binary system (or VLA4A) might be fed by an infalling streamer from envelope scales (~700 au). If this is the case, this streamer contributes to the accretion of material onto the system with a rate of at least 1.4x10-6 Msun yr-1. Conclusions. We conclude that the CH3CN emission in SVS13A traces hot gas from a complex structure. This complexity might be affected by a streamer that is possibly infalling and funneling material into the central region. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04495-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04495) | **$S^5$: Probing the Milky Way and Magellanic Clouds potentials with the  6-D map of the Orphan-Chenab stream**  |
|| Sergey E. Koposov, et al. -- incl., <mark>Ting S. Li</mark>, <mark>Lara R. Cullinane</mark>, <mark>Geraint F. Lewis</mark>, <mark>Sophia Lilleengen</mark> |
|*Appeared on*| *2022-11-10*|
|*Comments*| *submitted to MNRAS; comments welcome; data released with the paper is available on Zenodo this https URL*|
|**Abstract**| We present a 6-D map of the Orphan-Chenab (OC) stream by combining the data from 5 years of Southern Stellar Stream Spectroscopic Survey $S^5$ observations with Gaia EDR3 data. We reconstruct the proper motion, radial velocity, distance and on-sky track of stream stars with spline models and extract the stellar density along the stream. The stream has a total luminosity of $M_V=-8.2$ and an average metallicity of $[Fe/H]=-1.9$, similar to classical MW satellites like Draco. The stream shows drastic changes in its physical width varying from 200 pc to 1 kpc, a constant line of sight velocity dispersion of 5 km/s, but an increase in the velocity dispersion along the stream near pericenter to $\sim$ 10 km/s. Despite the large apparent variation in the stellar number density along the stream, the flow rate of stars along the stream is remarkably constant. We model the 6-D stream track by a Lagrange-point stripping method with a flexible MW potential in the presence of a moving extended LMC potential. This allows us to constrain the mass profile of the MW within the distance range 15.6 < r < 55.5 kpc, with the best measured enclosed mass of $(2.85\pm 0.1)\times10^{11}\,M_\odot$ within 32.4 kpc. With the OC stream's closest approach distance to the LMC of $\sim 21$ kpc, our stream measurements are highly sensitive to the LMC mass profile with the most precise measurement of the LMC's enclosed mass being at 32.8 kpc with $M=(7.02\pm 0.9)\times10^{10}\, {M}_\odot$. We confidently detect that the LMC DM halo extends to at least 53 kpc. The fitting of the OC stream allows us to constrain the past LMC trajectory and the degree of dynamical friction it experienced. We demonstrate that the stars on the OC stream show large energy and angular momentum spreads caused by the LMC perturbation and revealing the limitations of orbital invariants for substructure identification in the MW halo. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04472-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04472) | **Regular Solar Radio Imaging at Arecibo: Space Weather Perspective of  Evolution of Active Regions**  |
|| Periasamy K. Manoharan, et al. -- incl., <mark>Tapasi Ghosh</mark> |
|*Appeared on*| *2022-11-10*|
|*Comments*| *11 pages and 3 figures (White Paper Submitted to `Decadal Survey for Solar and Space Physics (Heliophysics) 2024-2033')*|
|**Abstract**| The sudden release of magnetic energy on the Sun drives powerful solar flares and coronal mass ejections. The key issue is the difficulty in predicting the occurrence time and location of strong solar eruptions, i.e., those leading to the high impact space weather disturbances at the near-Earth environment. Solar radio imaging helps identify the magnetic field characteristics of active regions susceptible to intense flares and energetic coronal mass ejections. Mapping of the Sun at X-band (8.1 -- 9.3 GHz) with the 12-m radio telescope at the Arecibo Observatory allows monitoring of the evolution of the brightness temperature of active regions in association with the development of magnetic complexity, which can lead to strong eruptions. For a better forecasting strategy in the future, such ground-based radio observations of high-spatial and temporal resolution, along with a full polarization capability, would have tremendous potential not only to understand the magnetic activity of solar eruptions, but also for revealing the particle acceleration mechanism and additional exciting science. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04777-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04777) | **A Study of Radio Knots within Supernova Remnant Cassiopeia A**  |
|| Xianhuan Lei, Hui Zhu, <mark>Haiyan Zhang</mark>, Wenwu Tian |
|*Appeared on*| *2022-11-10*|
|*Comments*| *20 pages,9 figures, ApJ accepted*|
|**Abstract**| The study on the dynamic evolution of young supernova remnants (SNRs) is an important way to understand the density structure of the progenitor's circumstellar medium. We have reported the acceleration or deceleration, proper motion and brightness changes of 260 compact radio features in the second youngest known SNR Cas A at 5\,GHz based on the VLA data of five epochs from 1987 to 2004. The radio expansion center locates at $\alpha(1950)=23^{\rm h}21^{\rm m}9^{\rm s}_{\cdot}7 \pm 0^{\rm s}_{\cdot}29, \delta(1950)=+58^{\circ}32^{\prime}25^{\prime\prime}_{\cdot}2 \pm 2^{\prime\prime}_{\cdot}2$. Three-quarters of the compact knots are decelerating, this suggests that there are significant density fluctuations in the stellar winds of the remnant's progenitor. We have verified that the acceleration or deceleration of compact knots are not related with the distribution of brightness. The brightening, fading, disappearing or new appearing of compact radio features in Cas A suggests that the magnetic field in the remnant is changing rapidly. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04808-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04808) | **A Multi-Wavelength View on the Rapidly-Evolving Supernova 2018ivc: An  Analog of SN IIb 1993J but Powered Primarily by Circumstellar Interaction**  |
|| Keiichi Maeda, et al. -- incl., <mark>Shiu-Hang Lee</mark> |
|*Appeared on*| *2022-11-10*|
|*Comments*| *31 pages, 14 figures, 3 tables. Accepted for publication in ApJ*|
|**Abstract**| SN 2018ivc is an unusual type II supernova (SN II). It is a variant of SNe IIL, which might represent a transitional case between SNe IIP with a massive H-rich envelope, and IIb with only a small amount of the H-rich envelope. However, SN 2018ivc shows an optical light curve evolution more complicated than canonical SNe IIL. In this paper, we present the results of prompt follow-up observations of SN 2018ivc with the Atacama Large Millimeter/submillimeter Array (ALMA). Its synchrotron emission is similar to that of SN IIb 1993J, suggesting that it is intrinsically an SN IIb-like explosion of a He star with a modest (~0.5 - 1 Msun) extended H-rich envelope. Its radio, optical, and X-ray light curves are explained primarily by the interaction between the SN ejecta and the circumstellar material (CSM); we thus suggest that it is a rare example (and the first involving the `canonical' SN IIb ejecta) for which the multi-wavelength emission is powered mainly by the SN-CSM interaction. The inner CSM density, reflecting the progenitor activity in the final decade, is comparable to that of SN IIb 2013cu that showed a flash spectral feature. The outer CSM density, and therefore the mass-loss rate in the final ~200 years, is larger than that of SN 1993J by a factor of ~5. We suggest that SN 2018ivc represents a missing link between SNe IIP and IIb/Ib/Ic in the binary evolution scenario. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04842-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04842) | **Modelling the magnetic structure of a large-scale horse-shoe-like  filament in a decaying and diffuse active region**  |
|| Kaifeng Kang, et al. -- incl., <mark>Yang Guo</mark>, <mark>Jun Lin</mark> |
|*Appeared on*| *2022-11-10*|
|*Comments*| *17 pages,14 figures,1 table*|
|**Abstract**| A large-scale, horse-shoe-like filament was investigated and the magnetic field around it was reconstructed. This is an intermediate filament (IF) that appeared on the solar disk for the first time at 02:00 UT on 2015 November 7, and took 8 days to move to the central median on the solar disk. The active region AR 12452 around which the filament occurred was diffuse so that the magnetic field nearby was weak, the average field strength is 106 G. Therefore, the existing approaches to extrapolating the coronal magnetic field and to constructing the filament configuration in the region with strong background field do not work well here. On the basis of the regularized Biot-Savart laws method, we successfully constructed a data-constrained, non-linear force-free field configuration for this IF observed on 2015 November 14. The overall IF configuration obtained in this way matches well the morphology suggested by a 304~\AA \ image taken by the Atmospheric Imaging Assembly on board Solar Dynamics Observatory. Magnetic dips in the configuration were coincident in space with the H$\alpha$ features of the filament, which is lower in altitude than the features seen in 304~\AA. This suggests that the cold plasma fills the lower part of the filament, and hot plasma is situated in the higher region. A quasi-separatrix layer wraps the filament, and both the magnetic field and the electric current are stronger near the inner edge of the filament. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04937-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04937) | **Relativistic contributions to Mars rotation**  |
|| Rose-Marie Baland, et al. -- incl., <mark>Aurélien Hees</mark> |
|*Appeared on*| *2022-11-10*|
|*Comments*| *15 pages, 7 figures, accepted for publication in A&A, InSight contribution ICN 304*|
|**Abstract**| Context: The orientation and rotation of Mars, which can be described by a set of Euler angles, is estimated from radioscience data and is then used to infer Mars internal properties. The data are analyzed using a modeling expressed within the Barycentric Celestial Reference System (BCRS). Aims: We provide new and more accurate (to the $0.1$ mas level) estimations of the relativistic corrections to be included in the BCRS model of the orientation and rotation of Mars to avoid a misinterpretation of the data. Methods: There are two types of relativistic contributions in Mars rotation and orientation: (i) those that directly impact the Euler angles and (ii) those resulting from the time transformation between a local Mars reference frame and BCRS. The former correspond essentially to the geodetic effect. We compute them assuming that Mars evolves on a Keplerian orbit. As for the latter, we compute the effect of the time transformation and compare the rotation angle corrections obtained using realistic orbits as described by ephemerides. Results: The relativistic correction in longitude comes mainly from the geodetic effect and results in the geodetic precession (6.754mas/yr) and the geodetic annual nutation (0.565 mas amplitude). For the rotation angle, the correction is dominated by the effect of the time transformation. The main annual, semi-annual, and ter-annual terms have amplitudes of 166.954 mas, 7.783 mas, and 0.544mas, respectively. The amplitude of the annual term differs by about 9 mas from the estimate usually considered by the community. We identify new terms at the Mars-Jupiter and Mars-Saturn synodic periods (0.567 mas and 0.102 mas amplitude) that are relevant considering the current level of uncertainty of the measurements, as well as a contribution to the rotation rate (7.3088 mas/day). There is no significant correction that applies to the obliquity. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2211.04474-b31b1b.svg)](https://arxiv.org/abs/arXiv:2211.04474) | **Planetary Exploration Horizon 2061 Report, Chapter 3: From science  questions to Solar System exploration**  |
|| Véronique Dehant, et al. -- incl., <mark>Heidi Hammel</mark> |
|*Appeared on*| *2022-11-10*|
|*Comments*| *107 pages, 37 figures, Horizon 2061 is a science-driven, foresight exercise, for future scientific investigations*|
|**Abstract**| This chapter of the Planetary Exploration Horizon 2061 Report reviews the way the six key questions about planetary systems, from their origins to the way they work and their habitability, identified in chapter 1, can be addressed by means of solar system exploration, and how one can find partial answers to these six questions by flying to the different provinces to the solar system: terrestrial planets, giant planets, small bodies, and up to its interface with the local interstellar medium. It derives from this analysis a synthetic description of the most important space observations to be performed at the different solar system objects by future planetary exploration missions. These observation requirements illustrate the diversity of measurement techniques to be used as well as the diversity of destinations where these observations must be made. They constitute the base for the identification of the future planetary missions we need to fly by 2061, which are described in chapter 4. Q1- How well do we understand the diversity of planetary systems objects? Q2- How well do we understand the diversity of planetary system architectures? Q3- What are the origins and formation scenarios for planetary systems? Q4- How do planetary systems work? Q5- Do planetary systems host potential habitats? Q6- Where and how to search for life? |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2211.05022.md
    + _build/html/tmp_2211.05022/./DCN_HFS_pSpe.png
    + _build/html/tmp_2211.05022/./Asymetry1.png
    + _build/html/tmp_2211.05022/./Spectrum_CASSIS.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# Based on observations carried out under project number L19MB with the IRAM NOEMA Interferometer. IRAM is supported by INSU/CNRS (France), MPG (Germany) and IGN (Spain):  and DCN

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2211.05022-b31b1b.svg)](https://arxiv.org/abs/2211.05022)<mark>Appeared on: 2022-11-10</mark> - _20 pages, 19 figures, accepted to A&A_

</div>
<div id="authors">

T.-H. Hsieh, et al. -- incl., <mark><mark>Th. Henning</mark></mark>, <mark><mark>G. V. Smirnov-Pinchukov</mark></mark>, <mark><mark>D. Semenov</mark></mark>, <mark><mark>B. Zhao</mark></mark>

</div>
<div id="abstract">

**Abstract:** Aims. We present high sensitivity and high-spectral resolution NOEMA observations of the Class 0/I binary system SVS13A, composed of the low-mass protostars VLA4A and VLA4B with a separation of ~90 au. VLA4A is undergoing an accretion burst that enriches the chemistry of the surrounding gas. This gives us an excellent opportunity to probe the chemical and physical conditions as well as the accretion process. Methods. We observe the (12K-11K) lines of CH3CN and CH313CN, the DCN (3-2) line, and the C18O (2-1) line toward SVS13A using NOEMA. Results. We find complex line profiles at disk scales which cannot be explained by a single component or pure Keplerian motion. By adopting two velocity components to model the complex line profiles, we find that the temperatures and densities are significantly different between these two components. This suggests that the physical conditions of the emitting gas traced via CH3CN can change dramatically within the circumbinary disk. In addition, combining our observations of DCN (3-2) with previous ALMA high-angular-resolution observations, we find that the binary system (or VLA4A) might be fed by an infalling streamer from envelope scales (~700 au). If this is the case, this streamer contributes to the accretion of material onto the system with a rate of at least 1.4x10-6 Msun yr-1. Conclusions. We conclude that the CH3CN emission in SVS13A traces hot gas from a complex structure. This complexity might be affected by a streamer that is possibly infalling and funneling material into the central region. 

</div>

<div id="div_fig1">

<img src="tmp_2211.05022/./DCN_HFS_pSpe.png" alt="Fig7" width="100%"/>

**Figure 7. -** 
DCN (3-2) centroid-velocity ({\it top left}) and linewidth ({\it top right}) maps from the blue-shifted component in the hyperfine structure fitting (see Appendix \ref{sec:decom}). The white contours show its peak intensity. The contour levels start at 25$\sigma$ and increase in 5$\sigma$ step, where $\sigma=0.3$ K. The blue contour shows the ALMA 1.3 mm continuum emission at the 3$\sigma$ level. ({\it Bottom}) The spectra of DCN (3--2), \ce{C^{18}O}(2--1), and \ce{CH3CN}($12_3--11_3$) toward the four positions marked in the right panel.
The blue bars show the relative intensities of the DCN (3--2) hyperfine structure at the velocity of the blue-shifted component (see Appendix \ref{sec:decom}).
 (*fig:dcn_fit*)

</div>
<div id="div_fig2">

<img src="tmp_2211.05022/./Asymetry1.png" alt="Fig10" width="100%"/>

**Figure 10. -** Zoom-in spectra for each K-ladder component of \ce{CH3CN} J$=$12-11. The orange curve shows the best-fit 1-component LTE model. The bottom frame in each panel shows the residual from the fitting. (*fig:asym*)

</div>
<div id="div_fig3">

<img src="tmp_2211.05022/./Spectrum_CASSIS.png" alt="Fig6" width="100%"/>

**Figure 6. -** Modeled \ce{CH3CN} and \ce{CH3 ^13CN} spectra overlaid on the observed spectra.
The first panel shows the LTE model one velocity component fitting. The second panel is the same as the first panel but treats \ce{CH3CN} and \ce{CH3 ^13CN} as two species with independent temperature, velocity and linewidth.
The third panel is the LTE two velocity component fitting while the fourth panel shows the non-LTE one.
The red curve shows the modeled spectrum;
The K=4 component of \ce{CH3CN} and K=3 component of \ce{CH3 ^13CN} suffer from severe line contamination so they were not included in the fitting. The model in these frequency ranges is shown with the dashed line.
The red and blue shaded areas in the two bottom panels represent the modeled spectrum from the individual components. The residual from the best-fit is shown in the bottom frame of each panel. The best-fit parameters are listed in Table \ref{tab:cassis} and the corner plots are shown in Appendix \ref{sec:mcmc}.
 (*fig:cassis*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

85  publications in the last 7 days.
	 _build/html/2211.05022.md
	 _build/html/2211.04048.md
	 _build/html/2211.03641.md
	 _build/html/2211.02502.md
	 _build/html/2211.01474.md
	 _build/html/2211.00667.md
	 _build/html/2211.00657.md
	 _build/html/2211.00087.md
	 _build/html/2211.00036.md
	 _build/html/2211.00020.md
	 _build/html/2211.00010.md
	 _build/html/2210.17434.md
	 _build/html/2210.12412.md
	 _build/html/2210.11926.md
	 _build/html/2210.11551.md
	 _build/html/2210.11103.md
	 _build/html/2210.10707.md
	 _build/html/2210.09930.md
	 _build/html/2210.08996.md
	 _build/html/2210.08517.md
	 _build/html/2210.08510.md
	 _build/html/2210.08078.md
	 _build/html/2210.06504.md
	 _build/html/2210.05695.md
	 _build/html/2210.05595.md
	 _build/html/2210.05505.md
	 _build/html/2210.04721.md
	 _build/html/2210.03615.md
	 _build/html/2210.01811.md
	 _build/html/2210.01809.md
	 _build/html/2210.01556.md
	 _build/html/2210.00863.md
	 _build/html/2210.00560.md
	 _build/html/2209.15403.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers